# Decision Trees

Decision Trees are models **used for regression and classification**. Here, we'll see the classification case, which  can be understood as a series of binary splits of data forming "branches" (each split representing a "node" usually known as **decision nodes**). These binary splits are performed in response to a "question" or **"tests"** with which we assess the data. The splits can be performed over both numerical and cathegorical data (which is an important 'plus' for this algorithm). The end nodes of a branch are known as **"leaf nodes"** and define a "prediction" for the value of a sample. To classify new values we analysed on which 'leaf node' it ends at and assign it a classification according to the classificiation most prevalent on that node.

A common way to desing a tree is to analyse the level of 'information' that we can get with different splits for the analised node. Notice that the method analyses one node at a time, hence, it is possible that a tree designed by finding the optimal split for each node is not the optimal tree overall. However, it must also be noted that analysing all the possible splits for the whole tree is computationally very expensive.

Now we need a criteria to split the tree. There are 2 common (and differentiable, thus easier for maximisation) ways to do this:

$$ \textrm {Gini index}= \sum_{k=1}^{K}p_{mk}(1 - p_{mk})$$ 

$$\textrm {Cross-Entropy (or just entropy)}= - \sum_{k=1}^{K}p_{mk}log_2(p_{mk})$$

where k = 1,..., K are the classification values, '*m*' is the node analysed and $p_{mk}$ is the proportion of data classified as *k* for node *m* (which can also be interpreted as the probability of class *k* on node *m*).

We'll continue using Entropy for this exercise. Notice that Entropy will be small when $p_{mk}$ is close to both 1 and 0. Interpreting *p* as a probability again, this would mean that **Entropy is smaller when the uncertainty is smaller** (*p*=1 and *p*=0 present zero uncertainty). Notice that this means that nodes lower on the tree (or children nodes) will most likely have lower entropy (since they will have a less diverse set of classes than the nodes before them -known as parent nodes) and that the starting node (or **root node**) is usually the one with the highest entropy.

In general terms, the Tree Creation algorithm follows these steps:

1) Calculate the information gain for **each possible split** of features and feature values (ie: the algorithm analyses all possibilities).\
2) Create a node splitting the data set using the feature and values for that feature that yield the **maximum information gain** for that split (information gain can be defined as the Entropy or the Gini index).\
3) Continue creating decision nodes/splits until a stopping criteria is reached. Common creteria are:\
    Maximum Depth\
    Minimum number of samples\
    Minimum impurity decrease
    
This is known as a **greedy approach** since it tries to maximise the entropy on every step (instead of trying to look at the whole tree for the best overall splits).

Growing a Decision Tree implies defining the "if/else" questions and their thresholds in order to get an answer as quickly as possible for new data.

Common problems with Decision Trees is their high variance when we use (even slightly) different data, which can trigger a different split on one or several nodes, as well as their proclivity to 'overfit' the model to a particular training data set which leaves us with a model that performs poorly on new data.

Common strategies to prevent overfitting are:
1) **Pre-pruning**: stopping the creation of more branches early by **limiting the depth** of the tree (number of levels)or requiring a minimum numer of samples in a node to continue splitting (notice that this is the method used in scikit-learn)\
2) **Post-pruning**: growing the tree completely and then prune it.

Finally, a key concept in Decision Trees is the **Information Gain** or reduction of entropy. As we've said, the root node has the highest entropy in a Tree and, given the greedy algorithm often used to grow it, subsequent nodes will dedice the entropy. The change in entropy between a (partent) node and the ones splitting from it (children) is known as information gain:

$$\textrm {Information Gain = Parent-Entropy - weighted-avg * Children-entropy}$$

Where the weighted average is defined by the number of samples at every children node.

We'll use 2 classes: Node and DecisionTreeClassifier.

To avoid overfitting, we'll use a min_number of samples to split (2) and a maximum depth (50). We will analyse the effect of increasing the max_depth later.

Notice that part of this code was inspired "Data Science from Scratch" by j. Grus and "AI and Machine Learning for Coders" by L. Moroney. It also uses examples from @AssemblyAI

**Class Node**: defines a node with attributes: index, threshold, left_branch and right_branch

**Class DecisionTreeClass** with methods:\

**_entropy**: returns the entropy of a node calculated as $\sum -{p}*{log(p)}$\

**_split**: separates a received sample X according to a received threshold, splitting the sample in "left" and "right" branches\

**_inf_gain**: calculates and returns information gain determined as parent_node_entropy - children_node_entropy (this is calculated for all possible splits\

**_best_split**: goes through all possible splits on a given node and uses **_inf_gain** for all of them to determine the one that **returns the feature** (index) **and the threshold** that define the best split (max inf_gain) for that node/

**_grow_tree**: this is the heart of the code. This method uses itself recursively and _best_split to generate the branches of the tree until it reaches a node . It returns a Node class, defined by its index (feature), threshold (that separates the branches that grow from that node) and the left-right branches (defined by Nodes as the method calls itself recursively)/

**_gothrough_tree**: starting from a given node, uses it's index feature and threshold to go through the tree until it finds an end node and **returns that end node's value**
    
**predict**: uses _gothrough_tree and a new sample X to find and end node and its value as a prediction
    
**fit**: uses _grow_tree from the root to create a tree using training samples X and values/classes y

In [20]:
import numpy as np
from collections import Counter

class Node():
    def __init__(self, index = None, threshold = None, left_branch = None, right_branch = None, info_gain = None, *, value = None):
        
        #decision nodes 
        self.feature_index = index
        self.threshold = threshold
        self.left = left_branch
        self.right = right_branch
        self.info_gain = info_gain
        self.value = value #Notice that value is only defined for leaf/terminal nodes 
        
    #for leaf or terminal nodes we need to define a value to define to ease the class definition
    def is_leaf_node(self):
        return self.value is not None

class DecisionTreeClass():
    def __init__(self, min_samples_split = 2, max_depth = 50, n_features = None):
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.n_features = n_features
        self.root = None

    def fit(self, X, y):
        #get characteristics of the [remaining] training data set (consider the min between the defined features and the given features)
        self.n_features = X.shape[1] if not self.n_features else min(X.shape[1], self.n_features)
        self.root = self._grow_tree(X,y)
        
    #this is the key method. It recursively create tree (with initial depth = 0)
    def _grow_tree(self, X, y, depth = 0):
        n_samples = X.shape[0]
        n_rem_labels = len(np.unique(y)) #records the different labels for the remaining samples in the training set
        
        #check stopping criteria: size of tree, only 1 label (entropy = 0)
        if (depth >= self.max_depth or n_rem_labels == 1 or n_samples <= self.min_samples_split):
            counter = Counter(y)
            leaf_value = counter.most_common(1)[0][0] #this is the label of the most common element in this terminal node
            return Node(value = leaf_value)
        
        #find the best split for the node. We shuffle the features. 
        #Notice that we can also select just a portion of the feature, this is useful for the random trees algorithm
        feat_indexes = np.random.choice(range(self.n_features), self.n_features, replace = False) 
        best_feat, best_thrsh  = self._best_split(X, y, feat_indexes)
        
        #create children nodes recursively slicing the sample
        left_indexes, right_indexes = self._split(X[:, best_feat], best_thrsh)
        left = self._grow_tree(X[left_indexes], y[left_indexes], depth + 1)
        right = self._grow_tree(X[right_indexes], y[right_indexes], depth + 1)
        return Node(best_feat, best_thrsh, left, right)
        
        
    #this function evaluates all possible splits and defines the one 
    #that maximises the information gain
    def _best_split(self, X, y, feat_indexes):
        best_gain = -10
        split_index, split_thrshld = None, None
        
        #we define all possible splits
        for f_indx in feat_indexes:
            X_col = X[:, f_indx] #selects all rows for the f_indx column
            thrshld = np.unique(X_col) #takes all the unique possible splits in the col
            
            #calculates inf gain for all combinations
            for thr in thrshld:
                inf_gain = self._inf_gain(y, X_col,thr)
                
                #update the register for the best gain split
                if inf_gain > best_gain:
                    best_gain = inf_gain 
                    split_index = f_indx
                    split_thrshld = thr
        
        return split_index, split_thrshld

    ##calculates and returns information gain for a given threshold to split on a node    
    def _inf_gain(self, y, X_col, thr):
        #parent's entropy
        parent_entropy = self._entropy(y)
        
        #define children's branches by indixes
        left_branch, right_branch = self._split(X_col, thr)
        
        #if there's no more splitting to do, inf. gain is null
        if len(left_branch) == 0 or len(right_branch) == 0:
            return 0
        
        #define children's entropy
        n_left, n_right = len(left_branch), len(right_branch)
        entropy_left, entropy_right = self._entropy(y[left_branch]), self._entropy(y[right_branch])
        children_entropy = (n_left/len(y))*entropy_left + (n_right/len(y))*entropy_right
        
        #calculate information gain
        inf_gain = parent_entropy - children_entropy
        return inf_gain
        
    #returns arrays with left-right branches according to a threshold value and a feature (X_col)
    def _split(self, X_col, split_thr):
        left_child = np.argwhere(X_col <= split_thr).flatten() #we use flatten() to have just one array
        right_child = np.argwhere(X_col > split_thr).flatten() #we use flatten() to have just one array
        return left_child, right_child
    
    #calculate entropy for a given sample set of labels
    def _entropy(self, y):
        labels, labels_count = np.unique(y, return_counts = True)
        prop_label = labels_count / len(y)
        return -np.sum([p*np.log(p) for p in prop_label if p > 0])
    
    #return an array of the predictions for an array of values    
    def predict(self, X):
        return np.array([self._gothrough_tree(x, self.root) for x in X])
        

    #recursively determine the end_node for a given value and return its most prominent label(prediction)
    def _gothrough_tree(self, x, node):

        #for terminal node, return its value
        if node.is_leaf_node():
            return node.value
        
        if x[node.feature_index] <= node.threshold:
            return self._gothrough_tree(x, node.left)
        return self._gothrough_tree(x, node.right)

To test the code I'll use the openly available 'iris.csv' database. 

Using sklearn to split the data into training and testing, we can find a 93% accuracy with 50 levels of depth and 2 min_split size for the tree. Notice that the value decreases to 90% if we increase the depth to 500 and the min_split increases to 20.

In [1]:
import numpy as np
import pandas as pd

col_names = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'type']
data = pd.read_csv("data/Iris.csv", skiprows = 1, header = None, names = col_names)
data.head(10)

data.shape

(150, 5)

In [21]:
#evaluate accuracy
def accuracy(y_test, y_prediction):
    return np.sum(y_test == y_prediction) / len(y_test)

X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=41)


dt = DecisionTreeClass(min_samples_split = 20, max_depth = 500, n_features = None)

dt.fit(X_train, y_train)

predictions = dt.predict(X_test)

print(accuracy(y_test, predictions))

0.9
